In [1]:
from data_pipeline import DataGen
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.utils import class_weight

Set up the dataset from our generator:

In [2]:
batch_size = 16
tracks = pd.read_csv('./data/processed_genres.csv')
train_gen = DataGen(tracks, batch_size=batch_size, shuffle=True)
output_signature=(
  tf.TensorSpec(shape=(512, 512), dtype=tf.float32, name=None),
  tf.TensorSpec(shape=(), dtype=tf.int32, name=None))
main_ds = tf.data.Dataset.from_generator(train_gen, output_signature=output_signature)

data = main_ds.cache()
data = data.shuffle(buffer_size=1000)
data = data.batch(batch_size)
data = data.prefetch(8)

train = data.take(36)
test = data.skip(36).take(15)

Compute the class weights for balancing:

In [3]:
genres = np.array(tracks['parent_genre_id'])
class_weights = class_weight.compute_class_weight(class_weight='balanced',
                                                  classes=np.unique(genres),
                                                  y=genres)

class_weights = dict(enumerate(class_weights))

class_weights

{0: 0.7104138166894665,
 1: 1.3136066610455313,
 2: 0.21967533840947548,
 3: 1.0263092885375493,
 4: 0.6930326957295374,
 5: 4.05712890625,
 6: 0.24705637488106566,
 7: 1.550186567164179,
 8: 10.116477272727273,
 9: 21.05320945945946,
 10: 13.202860169491526,
 11: 8.75245786516854,
 12: 2.533231707317073,
 13: 3.205632716049383,
 14: 1.1548832468495183,
 15: 74.1875}

Build and train the model:

In [4]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (16,16), input_shape=(512, 512, 1), activation="relu"),
    tf.keras.layers.MaxPooling2D((4, 4)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv2D(32, (16, 16), activation="relu"),
    tf.keras.layers.MaxPooling2D((4, 4)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(32, activation="relu"), 
    tf.keras.layers.Dense(16)
])
    
model.summary()
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=["accuracy"], optimizer='adam')

history = model.fit(x=train, epochs=50, validation_data=test, class_weight=class_weights)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 497, 497, 16)      4112      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 124, 124, 16)     0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 124, 124, 16)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 32)      131104    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 27, 27, 32)       0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 27, 27, 32)        0

UnknownError: Graph execution error:

InvalidArgumentError: {{function_node __wrapped__IO>AudioReadableRead_device_/job:localhost/replica:0/task:0/device:CPU:0}} read 264600 from 1188551 failed: 0 [Op:IO>AudioReadableRead]
Traceback (most recent call last):

  File "C:\Users\Peter\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tensorflow\python\ops\script_ops.py", line 271, in __call__
    ret = func(*args)

  File "C:\Users\Peter\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tensorflow\python\autograph\impl\api.py", line 642, in wrapper
    return func(*args, **kwargs)

  File "C:\Users\Peter\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tensorflow\python\data\ops\dataset_ops.py", line 1039, in generator_py_func
    values = next(generator_state.get_iterat